# Glucose Histogram

This notebook generates a histogram showing the distribution of glucose levels from blood glucose readings held in the Health Tracker database and retrieved via the Health Tracker Web Service.

- Histogram and KDE chart
- Faceted histogram and KDE charts for time of day (morning, afternoon, evening and night)

KDE plots provide a "smooth curve" over histogram data.

Before attempting to run the notebook:

- Make sure the variables defined in "config.ipynb" are set correctly
- Set the reporting date range and export options in the first code cell

In [ ]:
from datetime import date, timedelta, datetime

# Retrieve all-time blood glucose readings
start = date(2025, 1, 1)
end = start + timedelta(days=11)

# Exclusion date ranges - this should be a list of tuples in which each tuple has two
# members, a start date and time and end date and time in that order
exclusions = [
]

# Export format for the chart:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
chart_export_format = "PNG"

In [ ]:
%run ../api.ipynb
%run ../config.ipynb
%run ../export.ipynb

In [ ]:
# Log in to the service, get the person ID and retrieve the data
token = authenticate(url, username, password)
person_id = get_person_id(url, token, firstnames, surname)
df = get_blood_glucose_measurements(url, token, person_id, start, end)

# Remove any excluded ranges
df = remove_records_for_date_ranges(df, exclusions)

# Preview the data
df.head()

In [ ]:
def categorize_time_of_day(hour):
    if 5 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 17:
        return "Afternoon"
    elif 17 <= hour < 21:
        return "Evening"
    else:
        return "Night"

# Extract the hour and categorise each reading by the time of day
df["hour"] = df["date"].dt.hour
df["time_of_day"] = df["hour"].apply(categorize_time_of_day)

# Preview the data
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))

# Histogram with density normalization. This ensures the 
sns.histplot(df["level"], color="orange", edgecolor="red", kde=False, stat="density")

# KDE chart
sns.kdeplot(df["level"], color="darkred", linewidth=2, fill=True, alpha=0.3)

# Set the chart properties
plt.title(f"Glucose Distribution : {start.strftime('%d-%b-%Y')} to {end.strftime('%d-%b-%Y')}")
plt.xlabel("Glucose (mmol/L)")
plt.ylabel("Frequency / Density")
plt.grid(True)
plt.tight_layout()

# Export to PNG or PDF, if required
export_chart("glucose", "glucose_histogram", None, chart_export_format)

# Display the chart
plt.show()

In [ ]:
import seaborn as sns

# Define the order in which the facets should be plotted
time_order = ["Morning", "Afternoon", "Evening", "Night"]

# Create a FacetGrid
g = sns.FacetGrid(df, col="time_of_day", col_order=time_order, col_wrap=2, sharex=True, sharey=True, height=4)

# Add histogram (normalized to density)
g.map_dataframe(sns.histplot, x="level", stat="density", color="orange", edgecolor="red")

# Add KDE with shaded area
g.map_dataframe(sns.kdeplot, x="level", color="darkred", fill=True, alpha=0.3, linewidth=2)

# Clean facet titles so they show e.g. Morning rather than time_of_day=Morning
for ax in g.axes.flatten():
    title = ax.get_title()
    new_title = title.split("=")[-1].strip()
    ax.set_title(new_title)

# Add titles and labels
g.set_axis_labels("Glucose (mmol/L)", "Density")
g.fig.suptitle(f"Glucose Distribution : {start.strftime('%d-%b-%Y')} to {end.strftime('%d-%b-%Y')}", fontsize=16)
g.fig.tight_layout()
g.fig.subplots_adjust(top=0.9)


# Export to PNG or PDF, if required
export_chart("glucose", "glucose_histogram", "TimeOfDay", chart_export_format)
